In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# import data
filename_results = '/Users/chrismader/Python/SLDS/Output/gridsearch_results.csv'
filename_static = '/Users/chrismader/Python/SLDS/Data/static_data.csv'
res = pd.read_csv(filename_results)
static = pd.read_csv(filename_static, usecols=[0,1,2])

In [ ]:
# add sectors, cpll_gap
res['sector'] = res['security'].map(static.set_index('Ticker')['SectorID'])
res.insert(res.columns.get_loc('security') + 1, 'sector', res.pop('sector'))

In [ ]:
res.columns  #.to_list()

In [ ]:
# selections
n_regimes_sel = [3, 4]         # rows
sectors_sel   = ['CD', 'CS']   # columns

res_f = res.loc[
    res['n_regimes'].isin(n_regimes_sel) & res['sector'].isin(sectors_sel),
    ['config','n_regimes','dim_latent','sector','score']]

# pivot on filtered data
pt = res_f.pivot_table(
    index=['config','n_regimes','dim_latent'],
    columns='sector',
    values='score',
    aggfunc='mean').sort_index().sort_index(axis=1)

# drop rows with any inf in selected sectors, then row-mean
finite_mask = np.isfinite(pt.to_numpy()).all(axis=1)
pt_sel = pt.loc[finite_mask].copy()
pt_sel['avg'] = pt_sel.mean(axis=1, skipna=True)

# sort and style
pt_sel = pt_sel.sort_values('avg', ascending=False)
styled_sel = (
    pt_sel.style
         .format('{:.3f}')
         .background_gradient(cmap='RdYlGn', vmin=-0.1, vmax=0.1)
         .highlight_null())

styled_sel


In [ ]:
# spread of avg per config
spread = pt_sel.groupby('config')['avg'].agg(['mean','std','min','max'])
spread['range'] = spread['max'] - spread['min']

# sort by mean (descending)
spread = spread.sort_values('mean', ascending=False)

# style with colors on mean
styled_spread = (
    spread.style
          .format('{:.3f}')
          .background_gradient(subset=['mean'], cmap='RdYlGn', vmin=-0.1, vmax=0.1)
)
styled_spread

In [ ]:
# selections
n_regimes_sel = [3, 4]         # rows
sectors_sel   = ['CD', 'CS']   # columns
value_sel = 'cpll (max all runs)'

res_f = res.loc[
    res['n_regimes'].isin(n_regimes_sel) & res['sector'].isin(sectors_sel),
    ['config','n_regimes','dim_latent','sector', value_sel]]

# pivot on filtered data
pt = res_f.pivot_table(
    index=['config'],
    columns='sector',
    values=value_sel,
    aggfunc='mean').sort_index().sort_index(axis=1)

# drop rows with any inf in selected sectors, then row-mean
finite_mask = np.isfinite(pt.to_numpy()).all(axis=1)
pt_sel = pt.loc[finite_mask].copy()
pt_sel['avg'] = pt_sel.mean(axis=1, skipna=True)

# sort and style
pt_sel = pt_sel.sort_values('avg', ascending=False)
styled_sel = (
    pt_sel.style
         .format('{:.3f}')
         .background_gradient(cmap='RdYlGn', vmin=-50000, vmax=50000)
         .highlight_null())

styled_sel

In [ ]:
# selections
n_regimes_sel = [3, 4]         # rows
sectors_sel   = ['CD', 'CS']   # columns
value_sel = 'elbo_delta (max all runs)'

res_f = res.loc[
    res['n_regimes'].isin(n_regimes_sel) & res['sector'].isin(sectors_sel),
    ['config','n_regimes','dim_latent','sector', value_sel]]

# pivot on filtered data
pt = res_f.pivot_table(
    index=['config'],
    columns='sector',
    values=value_sel,
    aggfunc='mean').sort_index().sort_index(axis=1)

# drop rows with any inf in selected sectors, then row-mean
finite_mask = np.isfinite(pt.to_numpy()).all(axis=1)
pt_sel = pt.loc[finite_mask].copy()
pt_sel['avg'] = pt_sel.mean(axis=1, skipna=True)

# sort and style
pt_sel = pt_sel.sort_values('avg', ascending=False)
styled_sel = (
    pt_sel.style
         .format('{:.3f}')
         .background_gradient(cmap='RdYlGn', vmin=-50000, vmax=50000)
         .highlight_null())

styled_sel

In [ ]:
# --------------------------------------------------------------------------------------
# CONFIG
# --------------------------------------------------------------------------------------

PATHS = {
    "data_excel": "/Users/chrismader/Python/SLDS/Data/bbg_data.xlsx",
    "ff_dir": "/Users/chrismader/Python/SLDS/Data/",
    "ff_files": {
        "ff5": "F-F_Research_Data_5_Factors_2x3_daily.csv",
        "ff3": "F-F_Research_Data_Factors_daily.csv",
        "mom": "F-F_Momentum_Factor_daily.csv",},
    "results_csv": "/Users/chrismader/Python/SLDS/Out/gridsearch_results.csv",
    "segments_parquet": "/Users/chrismader/Python/SLDS/Out/gridsearch_segments.parquet",
    "tmp_dir":          "/Users/chrismader/Python/SLDS/tmp_slds/",
    "segments_tmp_csv": "/Users/chrismader/Python/SLDS/tmp_slds/segments_tmp.csv",
}

CONFIG = {
    
    # Core defaults
    "n_jobs": -1,  # multi-threading
    "dt": 1.0 / 252.0,
    "n_iters": 50,
    "h_z": 3.0,  # CUSUM parameter
    
    # Batch windows
    "batch_grid": [
        {"train_window": 756, "overlap_window": 5},
        # {"train_window": 256, "overlap_window": 63},
        # {"train_window": 504, "overlap_window": 63},
        # {"train_window": 756, "overlap_window": 63},
        # {"train_window": 1260, "overlap_window": 63},
    ],

    # Number of regimes
    # "K_grid": [2, 3, 4],
    "K_grid": [3],
    
    # Unrestricted models: 
    "unrestricted_models": [
        # {"label": "[y]",         "channels": ["y"],                "dim_latent": [1]},
        # {"label": "[y,h]",       "channels": ["y","h"],            "dim_latent": [2]},
        # {"label": "[g,v]",       "channels": ["g","v"],            "dim_latent": [2]},
        # {"label": "[g,v,h]",     "channels": ["g","v","h"],        "dim_latent": [2,3]},
        # {"label": "[y,g,v,h]",   "channels": ["y","g","v","h"],    "dim_latent": [3,4]},
    ],

    # Restricted models: 
    "restricted_models": [
        # {"label": "fund1",        "channels": ["y"],                 "dim_latent": [2],    "C_type": "fund1"},
        # {"label": "fund1_vix",    "channels": ["y","h"],             "dim_latent": [3],    "C_type": "fund1_vix"},
        # {"label": "fund2",        "channels": ["y","g"],             "dim_latent": [2],    "C_type": "fund2"},
        # {"label": "fund2_vix",    "channels": ["y","g","h"],         "dim_latent": [3],    "C_type": "fund2_vix"},
        # {"label": "fund3",        "channels": ["y","v","g"],         "dim_latent": [2],    "C_type": "fund3"},
        # {"label": "fund3_vix",    "channels": ["y","v","g","h"],     "dim_latent": [3],    "C_type": "fund3_vix"},   

        # {"label": "factor1",      "channels": ["y"],                 "dim_latent": [2],    "C_type": "factor1"},
        # {"label": "factor1_vix",  "channels": ["y","h"],             "dim_latent": [3],    "C_type": "factor1_vix"},

        {"label": "factor2_ff3",   "channels": ["y","mkt","smb","hml"],                   "dim_latent": [3], "C_type": "factor2"},
        # {"label": "factor2_ff3mom","channels": ["y","mkt","smb","hml","mom"],             "dim_latent": [4], "C_type": "factor2"},
        # {"label": "factor2_ff5",   "channels": ["y","mkt","smb","hml","rmw","cma"],       "dim_latent": [5], "C_type": "factor2"},
        # {"label": "factor2_ff5mom","channels": ["y","mkt","smb","hml","rmw","cma","mom"], "dim_latent": [6], "C_type": "factor2"},
    ],

    # Model selection
    "run_unrestricted": False,
    "run_restricted": True,

    # Output
    "verbose": False,
    "display": False,
}

for k, v in PATHS.items(): 
    CONFIG[k] = v
# per-security temp file templates used by IOManager
CONFIG["tmp_results_fmt"]  = "{tmp_dir}/tmp_res_{security}.csv"
CONFIG["tmp_segments_fmt"] = "{tmp_dir}/tmp_seg_{security}.csv"

In [ ]:
securities = list(res.security.unique())
filename_data = '/Users/chrismader/Python/SLDS/Data/bbg_data.xlsx'
# seed_stability_from_config(CONFIG, securities, filename_data)

In [ ]:
sel = ['config', 'dt', 'n_regimes', 'dim_latent', 'single_subspace', 'train_window', 'overlap_window',]
combos = res[sel].reset_index(drop=True)
print('config=', list(combos.config.unique()))
print('n_regimes=', list(combos.n_regimes.unique()))
print('dim_latent=', list(combos.dim_latent.unique()))
print('dt=', '1/252')
print('single_subspace=', 'True')
print('train_window=', 756)
print('overlap_window=', 5)

In [ ]:
def collect_seed_rolling_params(cfg, securities, n_runs=10, master_seed=123):
    """
    PURE WRAPPER: per security → rolling batches → per-seed fits.
    Uses only existing functions from your modules for fitting & evaluation.
    """
    import numpy as np  # needed for np.log, np.ones_like, etc.

    # existing helpers
    from gridsearch import import_data, import_factors, _intersect_indexes, _build_restrictions, _clean_elbo_runs
    from rSLDS import (
        fit_rSLDS, fit_rSLDS_restricted, inference_rSLDS, cusum_overlay,
        compute_smoothed_cpll, max_cpll_upper_bound, evaluate_rSLDS_actual)

    # pick active model strictly from cfg flags/lists
    if cfg["run_restricted"]:
        m = cfg["restricted_models"][0]; restricted = True; C_type = m["C_type"]
    else:
        m = cfg["unrestricted_models"][0]; restricted = False; C_type = None

    channels   = list(m["channels"])
    D          = int(m["dim_latent"][0])
    K          = int(cfg["K_grid"][0])
    dt         = float(cfg["dt"])
    n_iters    = int(cfg["n_iters"])
    h_z        = float(cfg["h_z"])
    batch_conf = cfg["batch_grid"][0]
    train_window = int(batch_conf["train_window"])
    overlap      = int(batch_conf["overlap_window"])

    # --- concise config debug ---
    print(f"[CFG] restricted={restricted} C_type={C_type} channels={channels} K={K} D={D} "
          f"dt={dt} n_iters={n_iters} h_z={h_z} train_window={train_window} overlap={overlap} n_runs={n_runs}")

    # data
    px_all, eps_all, pe_all, ser_vix = import_data(cfg["data_excel"])
    ff = import_factors(cfg["ff_dir"], cfg["ff_files"])

    def _series_by_key_for(sec):
        ser_px  = px_all[sec].dropna()
        ser_eps = eps_all[sec].dropna().where(lambda s: s > 0).dropna()
        ser_pe  = (ser_px / ser_eps).where(lambda s: s > 0).dropna()
        series = {
            "y": np.log(ser_px).diff().dropna(),
            "g": np.log(ser_eps).diff().dropna(),
            "v": np.log(ser_pe).diff().dropna(),
            "h": np.log(ser_vix).diff().dropna(),
        }
        for col, key in [("MKT","mkt"),("SMB","smb"),("HML","hml"),("RMW","rmw"),("CMA","cma"),("MOM","mom")]:
            if col in ff.columns:
                series[key] = ff[col]
        return series, ser_px

    rng = np.random.RandomState(master_seed)
    out = {}

    for sec in securities:
        print(f"\n[SEC] {sec}")
        series_by_key, ser_px = _series_by_key_for(sec)
        need = [series_by_key[k].dropna() for k in channels]
        common_idx = _intersect_indexes(need)
        if common_idx is None or getattr(common_idx, "empty", False):
            print(f"[WARN] {sec}: no overlap for channels {channels} — skipping")
            continue

        Y_full  = np.concatenate([series_by_key[k].loc[common_idx].values.reshape(-1,1) for k in channels], axis=1)
        px_full = ser_px.loc[common_idx].astype(float)
        T, N = Y_full.shape

        # sanity on N
        assert N == len(channels), f"[ASSERT] N={N} must equal len(channels)={len(channels)}"

        print(f"[DATA] T={T} N={N} idx[{common_idx[0]} → {common_idx[-1]}]")

        # batch schedule
        t0s, t1s, t0 = [], [], 0
        while t0 < T:
            t1s.append(min(t0 + train_window, T))
            t0s.append(t0)
            if t1s[-1] == T: break
            t0 += train_window - overlap

        print(f"[BATCH] batches={len(t0s)}")

        rec = {
            "meta": {
                "channels": channels, "K": K, "D": D,
                "train_window": train_window, "overlap_window": overlap,
                "common_index": common_idx,},
            "batches": []}

        params_spec = dict(n_regimes=K, dim_latent=D, single_subspace=True)

        for bi, (b0, b1) in enumerate(zip(t0s, t1s)):
            Yb = Y_full[b0:b1, :]
            pxb = px_full.iloc[b0:b1]
            idx_slice = pxb.index

            print(f"[B{bi}] slice t0={b0} t1={b1} len={len(idx_slice)}")

            restrictions = None
            if restricted:
                restrictions = _build_restrictions(C_type, Y_obs=Yb, base_channels=channels, D=D)
                assert "C" in restrictions and "d" in restrictions, "[ASSERT] restrictions must contain 'C' and 'd'"
                print(f"[B{bi}] restrictions keys={list(restrictions.keys())}")

            seeds = rng.randint(1, 2**31 - 1, size=n_runs).tolist()
            per_seed_records = []

            for si, s in enumerate(seeds):
                if si == 0:
                    print(f"[B{bi}] seed[0]={s} (of {n_runs})")

                if restricted:
                    xhat, zhat_raw, elbo, q, mdl = fit_rSLDS_restricted(
                        Yb, params_spec,
                        C=restrictions["C"], d=restrictions["d"],
                        n_iter_em=n_iters, seed=s,
                        b_pattern=restrictions.get("b_pattern"),
                        enforce_diag_A=True,
                        C_mask=restrictions.get("C_mask"),
                        d_mask=restrictions.get("d_mask"))
                else:
                    xhat, zhat_raw, elbo, q, mdl = fit_rSLDS(Yb, params_spec, n_iter_em=n_iters, seed=s)

                # basic asserts on core outputs
                assert hasattr(mdl, "dynamics") and hasattr(mdl, "emissions") and hasattr(mdl, "transitions"), \
                    "[ASSERT] fitted model missing required components"

                # evaluation path identical to gridsearch_actual
                mask_valid = np.ones_like(Yb, dtype=bool)
                gamma_valid, *_ = mdl.expected_states(xhat, Yb, mask=mask_valid)
                cpll = compute_smoothed_cpll(mdl, xhat, Yb, gamma_valid)
                max_cpll = max_cpll_upper_bound(Yb, reg_scale=1e-8)

                y_cus = Yb[:, 0].ravel()
                zhat_cusum = cusum_overlay(pxb, y_cus, xhat, mdl, h_z)

                elbo_clean = _clean_elbo_runs([elbo])

                summary = evaluate_rSLDS_actual(
                    Yb, pxb, zhat_cusum, xhat, elbo_clean, mdl,
                    [float(cpll)], [float(max_cpll)], dt, display=cfg["display"])

                # === PARAMS: extracted exactly like print_rSLDS_matrices ===
                Cs      = np.array(mdl.emissions.Cs,       copy=True)  # (1,N,D) or (K,N,D) or (N,D)
                ds      = np.array(mdl.emissions.ds,       copy=True)  # (1,N)   or (K,N)   or (N,)
                inv_eta = np.array(mdl.emissions.inv_etas, copy=True)  # (K,N) or (1,N)
                As      = np.array(mdl.dynamics.As,        copy=True)  # (K,D,D)
                bs      = np.array(mdl.dynamics.bs,        copy=True)  # (K,D)
                sigmasq = np.array(mdl.dynamics.sigmasq,   copy=True)  # (K,D)
                Rs      = np.array(mdl.transitions.Rs,     copy=True)  # (K,D)
                r       = np.array(mdl.transitions.r,      copy=True)  # (K,)

                # first-seed, first-batch: print small shape summary
                if si == 0:
                    def _sh(x): return getattr(x, "shape", None)
                    print(f"[B{bi}] param shapes: Cs{_sh(Cs)} ds{_sh(ds)} inv_etas{_sh(inv_eta)} "
                          f"As{_sh(As)} bs{_sh(bs)} sigmasq{_sh(sigmasq)} Rs{_sh(Rs)} r{_sh(r)}")

                # --- dynamics / transitions sanity (strict) ---
                assert As.shape[0] == K, f"[ASSERT] As first dim {As.shape[0]} != K={K}"
                assert bs.shape[0] == K, f"[ASSERT] bs first dim {bs.shape[0]} != K={K}"
                assert sigmasq.shape[0] == K, f"[ASSERT] sigmasq first dim {sigmasq.shape[0]} != K={K}"
                assert Rs.shape[0] == K, f"[ASSERT] Rs first dim {Rs.shape[0]} != K={K}"
                assert r.shape[0]  == K, f"[ASSERT] r first dim {r.shape[0]}  != K={K}"

                # --- emissions may be regime-shared under single_subspace ---
                # Cs
                if Cs.ndim == 3:
                    assert Cs.shape[0] in (K, 1), \
                        f"[ASSERT] Cs first dim {Cs.shape[0]} not in (K={K},1); shape={Cs.shape}"
                    if Cs.shape[0] == 1:
                        print(f"[B{bi}] NOTE: Cs tied across regimes; shape={Cs.shape}")
                elif Cs.ndim == 2:
                    print(f"[B{bi}] NOTE: Cs regime-shared 2D; shape={Cs.shape}")
                else:
                    raise AssertionError(f"[ASSERT] Cs unexpected rank: {Cs.ndim} with shape {Cs.shape}")

                # ds
                if ds.ndim == 2:
                    assert ds.shape[0] in (K, 1), \
                        f"[ASSERT] ds first dim {ds.shape[0]} not in (K={K},1); shape={ds.shape}"
                    if ds.shape[0] == 1:
                        print(f"[B{bi}] NOTE: ds tied across regimes; shape={ds.shape}")
                elif ds.ndim == 1:
                    print(f"[B{bi}] NOTE: ds regime-shared 1D; shape={ds.shape}")
                else:
                    raise AssertionError(f"[ASSERT] ds unexpected rank: {ds.ndim} with shape {ds.shape}")

                # inv_etas
                if inv_eta.ndim == 2:
                    assert inv_eta.shape[0] in (K, 1), \
                        f"[ASSERT] inv_etas first dim {inv_eta.shape[0]} not in (K={K},1); shape={inv_eta.shape}"
                    if inv_eta.shape[0] == 1:
                        print(f"[B{bi}] NOTE: inv_etas tied across regimes; shape={inv_eta.shape}")
                elif inv_eta.ndim == 1:
                    print(f"[B{bi}] NOTE: inv_etas regime-shared 1D; shape={inv_eta.shape}")
                else:
                    raise AssertionError(f"[ASSERT] inv_etas unexpected rank: {inv_eta.ndim} with shape {inv_eta.shape}")

                per_seed_records.append({
                    "seed": int(s),
                    "params": {
                        "Cs": Cs, "ds": ds, "inv_etas": inv_eta,
                        "As": As, "bs": bs, "sigmasq": sigmasq, "Rs": Rs, "r": r,},
                    "outputs": {
                        "zhat_raw": zhat_raw, "zhat_cusum": zhat_cusum,
                        "xhat": xhat, "elbo": elbo, "cpll": float(cpll), "max_cpll": float(max_cpll),},
                    "summary": summary,})

            rec["batches"].append({
                "t0": int(b0), "t1": int(b1),
                "idx_slice": idx_slice,
                "seeds": seeds,
                "records": per_seed_records,})

        out[sec] = rec

    return out


In [ ]:
def plot_param_boxplots(rollpack, security,
                        which=("A_diag","b","sigmasq","R","r","C_y","d_y","inv_eta_y"),
                        dim_list=None, suptitle=None):
    """
    Visualization helper: for each requested param, make boxplots across seeds per batch.
    Uses only matplotlib; handles regime-shared emissions by broadcasting to K.
    """
    import numpy as np
    import matplotlib.pyplot as plt

    rec = rollpack[security]
    channels = rec["meta"]["channels"]
    K = rec["meta"]["K"] if "K" in rec["meta"] else rec["batches"][0]["records"][0]["params"]["As"].shape[0]
    D = rec["meta"]["D"] if "D" in rec["meta"] else rec["batches"][0]["records"][0]["params"]["As"].shape[1]
    I = len(rec["batches"])
    if dim_list is None:
        dim_list = list(range(D))

    # --- helpers ---
    def _to_K(arr, want_ndim, tail_shape):
        """
        Broadcast regime-shared arrays to K along axis 0.
        - arr shapes allowed for emissions: (K, ...), (1, ...), tail only (...).
        - want_ndim is 2 for (K,D) / (K,N) and 3 for (K,N,D).
        """
        a = np.asarray(arr)
        if a.ndim == want_ndim and a.shape[0] == K:
            return a
        if a.ndim == want_ndim and a.shape[0] == 1:
            return np.repeat(a, K, axis=0)
        # tail-only provided, add regime axis then tile
        if a.shape == tail_shape:
            a = a[None, ...]
            return np.repeat(a, K, axis=0)
        raise ValueError(f"Cannot broadcast shape {a.shape} to (K, {', '.join(map(str, tail_shape))}) with K={K}")

    def _collect(key):
        vals = []
        for b in rec["batches"]:
            lst = []
            for rcd in b["records"]:
                P = rcd["params"]
                if key == "A_diag":
                    A = P["As"]  # (K,D,D)
                    lst.append(np.stack([np.diag(A[k]) for k in range(A.shape[0])], axis=0))  # (K,D)
                elif key in ("b","sigmasq","R"):
                    # map to stored names
                    mapped = {"b": "bs", "sigmasq": "sigmasq", "R": "Rs"}[key]
                    lst.append(P[mapped])   # expected (K,D)
                elif key == "r":
                    lst.append(P["r"])      # (K,)
                elif key in ("C_y","d_y","inv_eta_y"):
                    if "y" not in channels:
                        lst.append(None)
                    else:
                        y_row = channels.index("y")
                        Cs = P["Cs"]        # (K,N,D) or (1,N,D) or (N,D)
                        ds = P["ds"]        # (K,N)   or (1,N)   or (N,)
                        inv = P["inv_etas"] # (K,N)   or (1,N)   or (N,)
                        # broadcast to K
                        CsK  = _to_K(Cs, 3, (Cs.shape[-2], Cs.shape[-1]))  # -> (K,N,D)
                        dsK  = _to_K(ds, 2, (ds.shape[-1],))               # -> (K,N)
                        invK = _to_K(inv, 2, (inv.shape[-1],))             # -> (K,N)
                        if key == "C_y":
                            lst.append(CsK[:, y_row, :])  # (K,D)
                        elif key == "d_y":
                            lst.append(dsK[:, y_row])     # (K,)
                        else:
                            lst.append(invK[:, y_row])    # (K,)
                else:
                    lst.append(None)
            # stack over seeds
            if lst[0] is None:
                vals.append(None)
            else:
                arr = np.stack(lst, axis=-1)  # (K,D,R) or (K,R)
                vals.append(arr)
        return vals  # list length I

    # figure layout
    keys = []
    for k in which:
        col = _collect(k)
        if any(v is not None for v in col):
            keys.append((k, col))

    n_sub = sum(1 if k in ("r","d_y","inv_eta_y") else len(dim_list) for k, _ in keys)
    fig, axes = plt.subplots(n_sub, 1, figsize=(12, max(3, 2*n_sub)), sharex=True)
    if n_sub == 1:
        axes = [axes]
    axi = 0
    xs = np.arange(I)
    width = 0.12
    offsets = (np.arange(K) - (K-1)/2.0) * (width*1.5)

    def _boxplot_1D(ax, batch_list, label):
        for k in range(K):
            per_batch = []
            for i in range(I):
                arr = batch_list[i]  # (K,R) or None
                if arr is None:
                    per_batch.append(np.array([]))
                else:
                    per_batch.append(arr[k, :])
            ax.boxplot(per_batch, positions=xs + offsets[k], widths=width, manage_ticks=False, patch_artist=False)
        ax.set_ylabel(label); ax.grid(True, alpha=0.3)

    def _boxplot_2D(ax, batch_list, label, d):
        for k in range(K):
            per_batch = []
            for i in range(I):
                arr = batch_list[i]  # (K,D,R) or None
                if arr is None:
                    per_batch.append(np.array([]))
                else:
                    per_batch.append(arr[k, d, :])
            ax.boxplot(per_batch, positions=xs + offsets[k], widths=width, manage_ticks=False, patch_artist=False)
        ax.set_ylabel(f"{label} (dim {d})"); ax.grid(True, alpha=0.3)

    for key, batch_list in keys:
        if key in ("r","d_y","inv_eta_y"):
            ax = axes[axi]; axi += 1
            _boxplot_1D(ax, batch_list, key)
        else:
            for d in dim_list:
                ax = axes[axi]; axi += 1
                _boxplot_2D(ax, batch_list, key, d)

    axes[-1].set_xlabel("batch index (i)")
    if suptitle is None:
        suptitle = f"{security} • K={K}, D={D}"
    fig.suptitle(suptitle)
    plt.tight_layout(rect=[0,0,1,0.96])
    return fig


In [ ]:
# from gridsearch import *
# from rSLDS import *

# 1) Inputs
securities   = ["AAPL","MSFT"]
n_runs       = 10
master_seed  = 20241004

# 2) Collect rolling params across seeds (per security)
rollpack = collect_seed_rolling_params(CONFIG, securities, n_runs=n_runs, master_seed=master_seed)

In [ ]:
# 3) Plot for one security
security="AAPL"
model="factor2_ff3"
fig = plot_param_boxplots(
    rollpack, security=security,
    which=("A_diag","b","sigmasq","R","r","C_y","d_y","inv_eta_y"),  # drop *_y if 'y' not in channels
    dim_list=None,                 # or [0] to show only latent dim 0
    suptitle=f"{security} • {model}")
fig.savefig(f"/Users/chrismader/Python/SLDS/Out/{security}_seed_stability_{model}.png",
            dpi=200, bbox_inches="tight")